## get measurement dataset

In [2]:
import pickle

probe_atlas_file = "../datasets/probes_atlas.pickle"

with open(probe_atlas_file, "rb") as f:
    probes_atlas = pickle.load(f)

anchors = {}
for probe, probe_description in probes_atlas.items():
    if probe_description['is_anchor']:
        anchors[probe] = probe_description

In [3]:
from random import randint

NB_ANCHOR = 5
target_anchors_index = [randint(0, len(anchors)) for _ in range(0,NB_ANCHOR)]

# get target dataset
target_anchors = {}
anchors_ip = list(anchors.keys())
for index in target_anchors_index:
    target_anchors[anchors_ip[index]] = anchors[anchors_ip[index]]

# get vantage points dataset
vp_anchors = {}
for probe, probe_description in anchors.items():
    if not probe in target_anchors:
        vp_anchors[probe] = probe_description

In [4]:
print("targets")
for index, (probe, probe_description) in enumerate(target_anchors.items()):
    print(f"target_ip: {probe} : {probe_description}")
print()
print("vps:")
for index, (probe, probe_description) in enumerate(vp_anchors.items()):
    print(f"target_ip: {probe} : {probe_description}")


target_anchor_file = "../datasets/target_anchors.pickle"
vp_anchor_file = "../datasets/vp_anchors.pickle"

with open(target_anchor_file, "wb") as f:
    pickle.dump(target_anchors, f)

with open(vp_anchor_file, "wb") as f:
    pickle.dump(vp_anchors, f)

targets
target_ip: 207.226.224.236 : {'id': 6784, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1095, 'longitude': 8.7015}
target_ip: 46.19.61.33 : {'id': 7176, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1605, 'longitude': 8.7405}
target_ip: 196.6.220.42 : {'id': 6882, 'is_anchor': True, 'country_code': 'KE', 'latitude': -1.3315, 'longitude': 36.8715}
target_ip: 130.59.80.2 : {'id': 6317, 'is_anchor': True, 'country_code': 'CH', 'latitude': 47.3805, 'longitude': 8.5475}
target_ip: 77.88.216.1 : {'id': 6829, 'is_anchor': True, 'country_code': 'UA', 'latitude': 48.4715, 'longitude': 35.0585}

vps:
target_ip: 2.58.52.152 : {'id': 6534, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1095, 'longitude': 8.6785}
target_ip: 5.1.66.5 : {'id': 6886, 'is_anchor': True, 'country_code': 'AT', 'latitude': 48.1905, 'longitude': 16.3515}
target_ip: 5.1.99.172 : {'id': 6520, 'is_anchor': True, 'country_code': 'CH', 'latitude': 46.5175, 'longitude': 6.6385}
target_ip: 5.2

## from ip target list select probes

In [5]:
from utils.helpers import distance

target_anchor_file = "../datasets/target_anchors.pickle"
vp_anchor_file = "../datasets/vp_anchors.pickle"

with open(target_anchor_file, "rb") as f:
    target_anchors = pickle.load(f)

with open(vp_anchor_file, "rb") as f:
    vp_anchors = pickle.load(f)

def get_closest_vp(target_ip: str, target_description: dict, vp_anchors: dict, nb_vp: int):

    distance_probes_target = []
    for vp_ip, vp_description in vp_anchors.items():

        p_lat = vp_description["latitude"]
        p_lon = vp_description["longitude"]

        t_lat = target_description["latitude"]
        t_lon = target_description["longitude"]

        d = distance(p_lat, t_lat, p_lon, t_lon)
        distance_probes_target.append((vp_ip, d))

    distance_probes_target = sorted(distance_probes_target, key = lambda d: d[1])
    closest_vp = [vp_ip for vp_ip, _ in distance_probes_target[:nb_vp]]

    # filter out the target ip
    for vp_ip in closest_vp:
        if target_ip in closest_vp:
            raise RuntimeError("the two datasets should not have IP addresses in common")

    return closest_vp

In [6]:
import pandas as pd

from pathlib import Path

NB_VP = 15
measurements = {}
for target_ip, target_description in target_anchors.items():

    # retrieve the ten closest vp for each targets
    measurements[target_ip] = get_closest_vp(target_ip, target_description, vp_anchors, NB_VP)

    print(f"traget ip: {target_ip}: {target_description}")
    for vp_ip in measurements[target_ip]:
        print(f"probe ip {vp_ip}: {vp_anchors[vp_ip]}")
    print()

traget ip: 207.226.224.236: {'id': 6784, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1095, 'longitude': 8.7015}
probe ip 185.32.187.206: {'id': 7184, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1095, 'longitude': 8.6815}
probe ip 2.58.52.152: {'id': 6534, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1095, 'longitude': 8.6785}
probe ip 217.243.20.18: {'id': 6724, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1095, 'longitude': 8.6785}
probe ip 34.89.240.90: {'id': 6708, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1115, 'longitude': 8.6775}
probe ip 85.197.82.113: {'id': 6558, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1115, 'longitude': 8.7395}
probe ip 45.76.86.163: {'id': 6561, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1175, 'longitude': 8.7405}
probe ip 81.91.160.93: {'id': 6857, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1205, 'longitude': 8.7395}
probe ip 46.101.130.201: {'id': 6967, 'i

## start measurement

In [ ]:
import pickle
from pathlib import Path

from utils.atlas_probing import RIPEAtlas

RIPE_ACCOUNT = "timur.friedman@sorbonne-universite.fr"
RIPE_KEY = "b3d3d4fc-724e-4505-befe-1ad16a70dc87"
NB_PACKETS = 5

driver = RIPEAtlas(RIPE_ACCOUNT, RIPE_KEY)

measurement_results = {}
dry_run = False
for target_ip in measurements:
    print(f"starting measurement for target {target_ip}")
    print(f"with {len(measurements[target_ip])} vp: {[vp_ip for vp_ip in measurements[target_ip]]}")

    # get anchor ids
    vp_ids = [
        vp_anchors[vp_ip]['id']
        for vp_ip in measurements[target_ip]
    ]
    # run measurement
    if dry_run:
        measurement_id, results = None, None
    else:
        measurement_id, results = driver.probe(target_ip, vp_ids, NB_PACKETS)

    measurement_results[target_ip] = {}
    measurement_results[target_ip]['measurement_id'] = measurement_id
    measurement_results[target_ip]['results'] = results

    print(f"measurement done, measurement id: {measurement_id}")
    print()

measurement_file = Path(".") / "../datasets/test_geoloc/10_probes_10_target.pickle"

with open(measurement_file, "wb") as f:
    pickle.dump(measurement_results, f)